<a href="https://colab.research.google.com/github/NULabTMN/cs6200-hw2/blob/main/PageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PageRank

In this assignment, you will compute PageRank on a collection of 469,235 web sites using the iterative version of the PageRank algorithm described in class for sparse graphs (NOT the power method with explicit matrix multiplication).

Consider the following directed graph:

![A directed link graph](https://ccs.neu.edu/home/dasmith/courses/cs6200/pagerank.jpg)

We can represent this graph as a collection of nodes, here, ordered pairs of node index and node name:

In [ ]:
small_nodes = [(0, 'A'),
              (1, 'B'),
              (2, 'C'),
              (3, 'D'),
              (4, 'E'),
              (5, 'F')]

and a collection of directed links, i.e., ordered pairs from source to target:

In [ ]:
small_edges = [
  (0, 1),
  (0, 2),
  (0, 5),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (2, 3),
  (2, 4),
  (3, 0),
  (3, 2),
  (3, 4),
  (3, 5),
  (4, 0),
  (5, 0),
  (5, 1),
  (5, 4)
]

We use integer identifiers for the nodes for efficiency. Note that, unlike this example, in a real web graph, not every page will have in-links, nor will every page have out-links.

## First Implementation and Test

\[10 points\] Implement the iterative PageRank algorithm. Test your code on the six-node example using the input representation given above.  Be sure that your code handles pages that have no in-links or out-links properly.  (You may wish to test on a few such examples.) In later parts of this assignment, depending on how you store the data, it may be convenient to use iterators rather than storing the data in memory.

In [ ]:
,# TODO: Implement PageRank, given nodes and edges, to start with a uniform
# distribution over nodes, run a fixed number of iterations, and
# return a distribution over nodes.
from collections import defaultdict

def page_rank_iter(nodes, edges, iterations=10):
  I = [] #initializes values of all vertices to 1/len(small_nodes)
  R = [0]*len(small_nodes)
  a = 0.15 #value of lambda from the book.
  for i in range(len(small_nodes)):
    I.append(1/len(small_nodes))
  #dictionary containing the links that point to a specific vertex/page
  dict_ = defaultdict(list)
  for i in small_edges:
    dict_[i[1]].append(i[0])
  #dictionary counting a count of outgoing links for each vertex. 
  dict_2 = {}
  for i in small_nodes:
    dict_2[i[0]] = 0
  for i in small_edges:
    dict_2[i[0]] += 1
  while iterations >= 1: #page rank calculatations start here. 
    for i in range(len(small_nodes)):
      R[i] = a/len(small_nodes)
    c = 0
    for i in small_nodes:
      if dict_2[i[0]] == 0: #handles nodes with no out-links
        R[c] = (R[c] + (1-a))/len(small_nodes)
      else:
        length = len(dict_[i[0]])
        sum = 0
        for j in range(length):
          sum += I[dict_[i[0]][j]]/dict_2[dict_[i[0]][j]]
        if sum != 0: #handles page rank for nodes with no in-links 
          R[c] = (R[c] + (1-a))*sum 
      c += 1
    iterations -= 1
    I = R
  return I

# Output PageRank on the toy graph at various points.
# Make sure your output has node number, name, and PageRank value.

page_rank = page_rank_iter(small_nodes, small_edges, 1)
print("After 1 iteration the Pageranks are:")
for i in range(len(page_rank)):
  print('Node number:', small_nodes[i][0],  'Node name: ', small_nodes[i][1],  'PageRank value: ', round(page_rank[i],3))
#page_rank = page_rank_iter(small_nodes, small_edges, 10)
page_rank = page_rank_iter(small_nodes, small_edges, 100)
print("After 100 iterations the Pageranks are:")
for i in range(len(page_rank)):
  print('Node number:', small_nodes[i][0],  'Node name: ', small_nodes[i][1],  'PageRank value: ', round(page_rank[i],3))


After 1 iteration the Pageranks are:
Node number: 0 Node name:  A PageRank value:  0.231
Node number: 1 Node name:  B PageRank value:  0.097
Node number: 2 Node name:  C PageRank value:  0.122
Node number: 3 Node name:  D PageRank value:  0.109
Node number: 4 Node name:  E PageRank value:  0.194
Node number: 5 Node name:  F PageRank value:  0.122
After 100 iterations the Pageranks are:
Node number: 0 Node name:  A PageRank value:  0.035
Node number: 1 Node name:  B PageRank value:  0.017
Node number: 2 Node name:  C PageRank value:  0.019
Node number: 3 Node name:  D PageRank value:  0.012
Node number: 4 Node name:  E PageRank value:  0.022
Node number: 5 Node name:  F PageRank value:  0.017


## PageRank on Web Crawl Data

\[20 points\] Download and unpack a list of `.edu` websites and the links among them from the [Common Crawl](https://commoncrawl.org/2017/05/hostgraph-2017-feb-mar-apr-crawls/) open-source web crawl. For the sake of brevity, the data record links among websites, not web pages. The information for nodes and links is the same as the toy example above.

In [ ]:
!wget https://ccs.neu.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
!gzip -df vertices-edu.txt.gz
!wget https://ccs.neu.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
!gzip -df edges-edu.txt.gz

--2021-10-24 02:10:58--  https://ccs.neu.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
Resolving ccs.neu.edu (ccs.neu.edu)... 52.70.229.197
Connecting to ccs.neu.edu (ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3703486 (3.5M) [application/x-gzip]
Saving to: ‘vertices-edu.txt.gz’

vertices-edu.txt.gz 100%[===================>]   3.53M  14.1MB/s    in 0.3s    

2021-10-24 02:10:58 (14.1 MB/s) - ‘vertices-edu.txt.gz’ saved [3703486/3703486]

--2021-10-24 02:10:59--  https://ccs.neu.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
Resolving ccs.neu.edu (ccs.neu.edu)... 52.70.229.197
Connecting to ccs.neu.edu (ccs.neu.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12829526 (12M) [application/x-gzip]
Saving to: ‘edges-edu.txt.gz’

edges-edu.txt.gz    100%[===================>]  12.23M  28.5MB/s    in 0.4s    

2021-10-24 02:11:00 (28.5 MB/s) - ‘edges-edu.txt.gz’ saved [12829526/128295

There should now be files `vertices-edu.txt` and `edges-edu.txt`.

In [ ]:
# TODO: Process the raw data into the same format as the simple graph.
# You may use a list of iterators 
vertices_ = open("vertices-edu.txt")
edges_ = open("edges-edu.txt")

small_nodes = []
small_edges = []
lines = vertices_.readlines()
for line in lines:
    line = line.split(' ')
    small_nodes.append((int(line[0]), line[1]))
print(small_nodes[:20])
lines = edges_.readlines()
for line in lines:
    line = line.split(' ')
    small_edges.append((int(line[0]), int(line[1])))
print(small_edges[:20])

[(0, 'edu.00zl5e\n'), (1, 'edu.06hxbt\n'), (2, 'edu.082ifc\n'), (3, 'edu.083mjs\n'), (4, 'edu.09xzrr\n'), (5, 'edu.0aoqqj\n'), (6, 'edu.0ax4el\n'), (7, 'edu.0c5fez\n'), (8, 'edu.0cosn2\n'), (9, 'edu.0dcdp8\n'), (10, 'edu.0esfct\n'), (11, 'edu.0jn6my\n'), (12, 'edu.0k2wrx\n'), (13, 'edu.0kmhdw\n'), (14, 'edu.0n8grz\n'), (15, 'edu.0nnoiq\n'), (16, 'edu.0sklakwua222\n'), (17, 'edu.0utah.art\n'), (18, 'edu.0v1ofu\n'), (19, 'edu.0w7zm8\n')]
[(386, 440), (19202, 1033), (103884, 2635), (342306, 7399), (8366, 8312), (8358, 8312), (8949, 8987), (8982, 8987), (8910, 8987), (9028, 8999), (8917, 8999), (9003, 8999), (9030, 8999), (8868, 8999), (9001, 8999), (9072, 8999), (8887, 8999), (9019, 8999), (9023, 8999), (8949, 8999)]


Refine your implementation of PageRank to test for numerical convergence. Specificially, at each iteration, calculate the [perplexity](https://en.wikipedia.org/wiki/Perplexity) of the PageRank distribution, where perplexity is defined as 2 raised to the [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) of the PageRank distribution, i.e., $2^{H(PR)}$. (Recall that we defined entropy when talking about data compression.) The maximum perplexity of a PageRank distribution will therefore be the number of nodes in the graph.

At each iteration, check the _change_ in perplexity. If the change is less than some threshold, you can stop.


In [ ]:
# TODO: Implement convergence testing in PageRank
# If you choose, you can share some subroutines with your first version.
# Print the change in perplexity at each iteration.
from collections import defaultdict
import math

def perplexity(I): #this function calculates the perplexity
    hpr = 0
    for page in I:
        hpr += page*(math.log2(page))
    return 2**hpr

def page_rank(small_nodes, small_edges, thersold = 0.00001):
  I = [] #initializes values of all vertices to 1/len(small_nodes)
  R = [0]*len(small_nodes)
  a = 0.15 #value of lambda from the book.
  perplexity_ = [0,thersold+0.00001] #initializing values so that first while loop can run
  for i in range(len(small_nodes)):
    I.append(1/len(small_nodes))
  #dictionary containing in-links to a specific vertex/page
  dict_ = defaultdict(list)
  for i in small_edges:
    dict_[i[1]].append(i[0])
  #dictionary keeping a count of outgoing links for each vertex. 
  dict_2 = {}
  for i in small_nodes:
    dict_2[i[0]] = 0
  for i in small_edges:
    dict_2[i[0]] += 1
  while perplexity_[1] - perplexity_[0] > thersold: #stop when change in perplexity drops below thersold
    for i in range(len(small_nodes)):
      R[i] = a/len(small_nodes)
    c = 0
    for i in small_nodes:
      if dict_2[i[0]] == 0: #handles page rank for nodes with no out-links
          R[c] = (R[c] + (1-a))/len(small_nodes)
      else:
        length = len(dict_[i[0]])
        sum = 0
        for j in range(length):
          sum += I[dict_[i[0]][j]]/dict_2[dict_[i[0]][j]]
        if sum != 0: #handles page rank for nodes with no in-links 
          R[c] = (R[c] + (1-a))*sum 
      c += 1
    I = R
    perplexity_[0] = perplexity_[1]
    perplexity_[1] = perplexity(I)
  return I

# Output PageRank on the toy graph at various points.
# Make sure your output has node number, name, and PageRank value.\

#print(page_rank_iter(small_nodes, small_edges, iterations=10))
#print(page_rank_iter(small_nodes, small_edges, 1))
#print(page_rank_iter(small_nodes, small_edges, 10))
page_rank = page_rank(small_nodes, small_edges, 0.00001) #passing nodes, edges and thersold value

for i in range(41000, 41020): #outputting page rank of some links
  print('Node number:', small_nodes[i][0],  'Node name: ', small_nodes[i][1],  'PageRank value: ', page_rank[i])

Node number: 41000 Node name:  edu.buffalo.groundwater
 PageRank value:  3.101093144317111e-08
Node number: 41001 Node name:  edu.buffalo.grow
 PageRank value:  5.2624544824358514e-08
Node number: 41002 Node name:  edu.buffalo.gsa
 PageRank value:  2.440424449580272e-07
Node number: 41003 Node name:  edu.buffalo.gse
 PageRank value:  7.538488699330069e-07
Node number: 41004 Node name:  edu.buffalo.gse.gsemedia
 PageRank value:  1.8114597582645132e-06
Node number: 41005 Node name:  edu.buffalo.gse.gsestudent3
 PageRank value:  1.8114597582645132e-06
Node number: 41006 Node name:  edu.buffalo.gse.gseweb
 PageRank value:  1.8114597582645132e-06
Node number: 41007 Node name:  edu.buffalo.gse.gsewebvm
 PageRank value:  1.8114597582645132e-06
Node number: 41008 Node name:  edu.buffalo.gse.gsewebwp
 PageRank value:  1.8114597582645132e-06
Node number: 41009 Node name:  edu.buffalo.gse.lis
 PageRank value:  1.8114597582645132e-06
Node number: 41010 Node name:  edu.buffalo.gse.shujaa
 PageRank 

## Link Analysis

\[20 points\] In this final section, you will compute some properties of the web-site graph you downloaded.

First, consider the _in-link count_ of a website, simply the number of web-sites pointing to it (including self-links). 

In [ ]:
# TODO: List the document ID, domain name, and in-link count of the 50 websites with the highest in-link count
from operator import itemgetter
in_links = []
for i in range(len(small_nodes)):
  in_links.append([])
  in_links[i].append(small_nodes[i][0])
  in_links[i].append(small_nodes[i][1])
  in_links[i].append(0)
for i in small_edges:
    in_links[i[1]][2] = in_links[i[1]][2] + 1
in_links = sorted(in_links, key=itemgetter(2), reverse=True)
for i in range(50): #outputting page rank of some links
  print('Document ID:', in_links[i][0],  'Document name: ', in_links[i][1],  'In-link count: ', in_links[i][2])

Document ID: 185524 Document name:  edu.mit.web
 In-link count:  4388
Document ID: 278032 Document name:  edu.stanford
 In-link count:  4021
Document ID: 244433 Document name:  edu.purdue.english.owl
 In-link count:  3531
Document ID: 140443 Document name:  edu.indiana
 In-link count:  3339
Document ID: 237176 Document name:  edu.princeton
 In-link count:  3251
Document ID: 64587 Document name:  edu.columbia
 In-link count:  3123
Document ID: 465503 Document name:  edu.yale
 In-link count:  2804
Document ID: 418623 Document name:  edu.utexas
 In-link count:  2622
Document ID: 383763 Document name:  edu.unc
 In-link count:  2592
Document ID: 197698 Document name:  edu.nap
 In-link count:  2494
Document ID: 439637 Document name:  edu.washington
 In-link count:  2291
Document ID: 373442 Document name:  edu.umich
 In-link count:  2281
Document ID: 440674 Document name:  edu.washington.depts
 In-link count:  2276
Document ID: 148945 Document name:  edu.jhu.muse
 In-link count:  2255
Documen

Then, use the PageRank values compute by your second implementation. Note that some websites will have both a high in-link count and PageRank.

In [ ]:
# TODO: List the document ID, domain name, and PageRank of the 50 websites with the highest PageRank.
highest_ranks = []
for i in range(len(page_rank)):
  highest_ranks.append([])
  highest_ranks[i].append(small_nodes[i][0])
  highest_ranks[i].append(small_nodes[i][1])
  highest_ranks[i].append(page_rank)
highest_ranks = sorted(highest_ranks, key=itemgetter(2), reverse=True)
for i in range(50): #outputting page rank of some links
  print('Document ID:', highest_ranks[i][0],  'Document name: ', highest_ranks[i][1],  "Pagerank: ", highest_ranks[i][2])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Finally, compute some summary statistics on this dataset.

In [ ]:
# TODO: Compute:
# - the proportion of websites with no in-links (i.e., source nodes);
dict_ = defaultdict(list)
for i in small_edges:
  dict_[i[1]].append(i[0])
c = 0
for i in small_nodes:
  if i[0] not in dict_.keys():
    c += 1
print("Proportion of websites with no in-links: ",round(c/len(small_nodes),3))

# - the proportion of websites with no out-links (i.e., sink nodes);
dict_2 = {}
for i in small_nodes:
  dict_2[i[0]] = 0
for i in small_edges:
  dict_2[i[0]] += 1
c= 0
for key in dict_2:
    if dict_2[key] == 0:
        c = c + 1 
print("Proportion of websites with no out-links: ",round(c/len(small_nodes),3))

# - the proportion of websites whose PageRank is higher than the initial uniform distribution.
intial_distribution = 1/len(small_nodes)
c - 0
for i in page_rank:
  if i > intial_distribution:
    c += 1
print("Proportion of websites whose PageRank is higher than the initial uniform distribution: ",round(c/len(small_nodes),3))


Proportion of websites with no in-links:  0.262
Proportion of websites with no out-links:  0.611
Proportion of websites whose PageRank is higher than the initial uniform distribution:  0.617
